In [17]:
%load_ext lab_black

In [64]:
from sadie.anarci import Anarci
from sadie.anarci.numbering import scheme_numbering
import pandas as pd

In [74]:
main_file = "../../tests/unit/anarci/fixtures/split_files/scfv_heavy_7.fasta"
anarci_api = Anarci(allowed_species=["human"])
results = anarci_api.run_file(main_file)

In [75]:
def _get_region(row, start, end, segment_name):
    with_segment = "".join(
        list(
            map(
                lambda x: x[-1],
                list(
                    filter(
                        lambda x: x[0] >= start and x[0] <= end,
                        list(
                            zip(
                                row["Numbering"],
                                row["Insertion"],
                                row["Numbered_Sequence"],
                            )
                        ),
                    )
                ),
            )
        )
    )
    without_segment = with_segment.replace("-", "")
    return pd.Series(
        {
            f"{segment_name}_gaps": with_segment,
            f"{segment_name}_no_gaps": without_segment,
        }
    )


def add_segment_regions(df):
    for group, sub_df in df.groupby(["scheme", "region_definition", "Chain"]):
        numbering = group[0]
        chain = {"H": "heavy", "l": "light"}[group[-1]]
        boundaries = group[1]
        numbering_lookup = scheme_numbering[numbering][chain][boundaries]
        return_frames = []
        for region in [
            "fwr1_aa",
            "cdr1_aa",
            "fwr2_aa",
            "cdr2_aa",
            "fwr3_aa",
            "cdr3_aa",
            "fwr4_aa",
        ]:
            _start = numbering_lookup[f"{region}_start"]
            _end = numbering_lookup[f"{region}_end"]
            sub_df = sub_df.join(
                df.apply(lambda x: _get_region(x, _start, _end, region), axis=1)
            )
        return_frames = [sub_df]
    segmented_df = pd.concat(return_frames).reset_index(drop=True)
    segmented_df["leader"] = segmented_df[["sequence", "seqstart_index"]].apply(
        lambda x: x[0][: x[1]], axis=1
    )
    segmented_df["tail"] = segmented_df[["sequence", "seqend_index"]].apply(
        lambda x: x[0][x[1] + 1 :], axis=1
    )
    return segmented_df

In [76]:
results = add_segment_regions(results)

In [77]:
results

,Id,sequence,domain_no,hmm_species,chain_type,e-value,score,seqstart_index,seqend_index,identity_species,v_gene,v_identity,j_gene,j_identity,Chain,Numbering,Insertion,Numbered_Sequence,scheme,region_definition,allowed_species,allowed_chains,fwr1_aa_gaps,fwr1_aa_no_gaps,cdr1_aa_gaps,cdr1_aa_no_gaps,fwr2_aa_gaps,fwr2_aa_no_gaps,cdr2_aa_gaps,cdr2_aa_no_gaps,fwr3_aa_gaps,fwr3_aa_no_gaps,cdr3_aa_gaps,cdr3_aa_no_gaps,fwr4_aa_gaps,fwr4_aa_no_gaps,leader,tail
0,2571_SAMPLE_canine-round-3-phage_MOLECULE_1019...,EVQLVESGGDLVAPSQSLSITCTVSGLSLTGNSISWVRQPPGRGLE...,0,dog,H,0.000000e+00,176.625,0,122,human,IGHV3-53*03,0.700195,IGHJ2*01,0.790039,H,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[E, V, Q, L, V, E, S, G, G, -, D, L, V, A, P, ...",imgt,imgt,human,"H,K,L",EVQLVESGG-DLVAPSQSLSITCTVS,EVQLVESGGDLVAPSQSLSITCTVS,GLSL----TGNS,GLSLTGNS,ISWVRQPPGRGLEWLGV,ISWVRQPPGRGLEWLGV,IWSS---GGT,IWSSGGT,DYTDAVK-GRFTISRDNAKNT---LYLQMNSLRTEDTAVYYC,DYTDAVKGRFTISRDNAKNTLYLQMNSLRTEDTAVYYC,AKIYYYDADYLHWYFDF,AKIYYYDADYLHWYFDF,WGPGTLVTISS,WGPGTLVTISS,,
1,2571_SAMPLE_canine-round-3-phage_MOLECULE_1090...,EVQLVESGGDLVKPDGSLRLSCVVSGLSLTSNSMSWVRQAPGRGLQ...,0,dog,H,0.000000e+00,177.750,0,122,human,IGHV3-66*01,0.689941,IGHJ2*01,0.790039,H,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[E, V, Q, L, V, E, S, G, G, -, D, L, V, K, P, ...",imgt,imgt,human,"H,K,L",EVQLVESGG-DLVKPDGSLRLSCVVS,EVQLVESGGDLVKPDGSLRLSCVVS,GLSL----TSNS,GLSLTSNS,MSWVRQAPGRGLQWVAV,MSWVRQAPGRGLQWVAV,IWSN---GGT,IWSNGGT,EYTDAVK-GRFTISKDNSKSQ---VFLKMNSLQTDDTARYYC,EYTDAVKGRFTISKDNSKSQVFLKMNSLQTDDTARYYC,ASIHYYDADYLHWYFDF,ASIHYYDADYLHWYFDF,WGPGTLVTISS,WGPGTLVTISS,,
2,2571_SAMPLE_feline-round-1-phage_MOLECULE_7236...,DVQLVESGGDLVKPGGSLRLTCATSGMSLTSNSLNWVRQVPGKGLQ...,0,cat,H,0.000000e+00,178.250,0,122,human,IGHV3-66*01,0.700195,IGHJ2*01,0.569824,H,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[D, V, Q, L, V, E, S, G, G, -, D, L, V, K, P, ...",imgt,imgt,human,"H,K,L",DVQLVESGG-DLVKPGGSLRLTCATS,DVQLVESGGDLVKPGGSLRLTCATS,GMSL----TSNS,GMSLTSNS,LNWVRQVPGKGLQWVAD,LNWVRQVPGKGLQWVAD,IWGN---GGT,IWGNGGT,YYADSVK-GRFTISKDNAENT---LYLEMNNLKTEDTATYYC,YYADSVKGRFTISKDNAENTLYLEMNNLKTEDTATYYC,ASIYYYDADYLHWYFDF,ASIYYYDADYLHWYFDF,WGHGTIVTKCP,WGHGTIVTKCP,,
3,2572_SAMPLE_feline-round-4-cell_MOLECULE_37618...,DVQLVESGGDVVELGGSRGLSRMSAGLSMTSNSMSWISQAPGKGLQ...,0,dog,H,0.000000e+00,157.250,0,122,human,IGHV3-66*01,0.689941,IGHJ2*01,0.569824,H,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[D, V, Q, L, V, E, S, G, G, -, D, V, V, E, L, ...",imgt,imgt,human,"H,K,L",DVQLVESGG-DVVELGGSRGLSRMSA,DVQLVESGGDVVELGGSRGLSRMSA,GLSM----TSNS,GLSMTSNS,MSWISQAPGKGLQWVAD,MSWISQAPGKGLQWVAD,IWNN---GGT,IWNNGGT,YYADSVK-GRFTISRDNAKNT---LYLQMNSLKTEDTATYYC,YYADSVKGRFTISRDNAKNTLYLQMNSLKTEDTATYYC,ASIYYYDAEYLHWYFDF,ASIYYYDAEYLHWYFDF,WGHGTIVTKCP,WGHGTIVTKCP,,
4,2572_SAMPLE_canine-round-4-cell_MOLECULE_61623...,EVQLVESGGDLVKPEGSLRLSCVVSGLSLTSNEMSWVRQAPGRGLQ...,0,dog,H,0.000000e+00,177.625,0,122,human,IGHV3-66*01,0.680176,IGHJ2*01,0.709961,H,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[E, V, Q, L, V, E, S, G, G, -, D, L, V, K, P, ...",imgt,imgt,human,"H,K,L",EVQLVESGG-DLVKPEGSLRLSCVVS,EVQLVESGGDLVKPEGSLRLSCVVS,GLSL----TSNE,GLSLTSNE,MSWVRQAPGRGLQWVAT,MSWVRQAPGRGLQWVAT,IWSN---GGT,IWSNGGT,DYTDAVK-GRFTISKDNSKSQ---VFLKMNSLQTDDTARYYC,DYTDAVKGRFTISKDNSKSQVFLKMNSLQTDDTARYYC,ASIYYYDADYLHWYMDF,ASIYYYDADYLHWYMDF,WGPGTLVTISS,WGPGTLVTISS,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2572_SAMPLE_feline-round-4-cell_MOLECULE_12940...,DVQLVESGGDVVELGGSRGLSRMSAGLSLTDNSMSWISQAPGKGLQ...,0,dog,H,0.000000e+00,154.625,0,122,human,IGHV3-20*04,0.669922,IGHJ2*01,0.640137,H,"[1, 2, 3, 4,